Here we create the first simple model without integer variables

In [5]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
from typing import Tuple

def construct_equalities(
        supply_volumes: np.ndarray,
        supply_prices: np.ndarray,
        gen_matrix: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    В этой функции мы формируем матрицу системы A_eq и вектор свободных членов b_eq для системы ограничений-равенств.

    :param supply_volumes: вектор объемов ступеней кривой предложения
    :param supply_prices: вектор цен ступеней кривой предложения
    :param gen_matrix: матрица с информацией по генераторам
    :return: кортеж с матрицей системы A_eq и вектором свободных членов b_eq
    """
    m, n = gen_matrix.shape[0], supply_volumes.shape[0]
    Q = np.tile(supply_volumes, (m, 1))
    # here we the last column of gen_matrix where prices are written
    gen_prices = gen_matrix[:, -1]
    """
    здесь мы формируем матрицу aplha_coeffs размерности m*n, значения в которой рассчитываются следующим образом:

    alpha_coeffs[i, j] = 0, если цена i-ого генератора меньше цены j-ой ступени кривой
    alpha_coeffs[i, j] = 1, если цена i-ого генератора больше или равна цене j-ой ступени кривой
    """
    alpha_coeffs = calculate_alpha_coeffs(gen_prices=gen_prices, supply_prices=supply_prices)

    D1 = (np.ones_like(alpha_coeffs) - alpha_coeffs) * Q
    D2 = alpha_coeffs * Q

    k, col_index = 2*m*n, 0
    A1, A2 = np.zeros((m, k)), np.zeros((m, k))
    A3 = np.zeros((2*m, 4*m))
    for gen_index in range(m):
        A1[gen_index, col_index:col_index+n] = D1[gen_index, :]
        A2[gen_index, col_index:col_index+n] = D2[gen_index, :]
        col_index += n
    b1 = gen_matrix[:, 0] - gen_matrix[:, 1]
    b2 = gen_matrix[:, 2] - gen_matrix[:, 0]
    A_eq = np.vstack((A1, A2))

    for gen_index in range(2*m):
        A3[gen_index, 2*gen_index:2*gen_index+2] = np.array([-1, 1])
    A_eq = np.hstack((A_eq, A3))
    b_eq = np.concatenate((b1, b2))
    return A_eq, b_eq

def construct_inequalities(
        supply_volumes: np.ndarray,
        supply_prices: np.ndarray,
        gen_matrix: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:

    m, n = gen_matrix.shape[0], supply_volumes.shape[0]

    k = 2*m*n
    A_ineq = np.vstack((
        np.hstack((np.eye(n*m), -1*np.eye(n*m))),
        np.hstack((-1*np.eye(n*m), np.eye(n*m)))
    ))
    b_ineq = np.concatenate((np.zeros(n*m), np.ones(n*m)))

    A1, A2, A3 = np.zeros((m, k)), np.zeros((m, k)), np.zeros((m, k))
    col_index = m*n
    beta_vector = np.zeros(n)
    beta_vector[supply_prices > 0] = 1
    for gen_index in range(m):
        A1[gen_index, col_index:col_index+n] = 1 - beta_vector
        A2[gen_index, col_index:col_index+n] = beta_vector
        A3[gen_index, col_index:col_index+n] = -1*np.ones(n)
        col_index += n
    A = np.vstack((A1, A2, A3))
    b = np.concatenate((np.ones(m), 2*np.ones(m),-1*np.ones(m)))
    A_ineq = np.vstack((A_ineq, A))
    b_ineq = np.concatenate((b_ineq, b))

    A_ineq = np.hstack((A_ineq, np.zeros((A_ineq.shape[0], 4*m))))

    # here we`re processing the case when sum of partial volumes that belong to generators exceeds the whole volume of this stage in the supply curve
    A = np.hstack((
        np.tile(np.eye(n), (1, m)), np.zeros((n, n*m)), np.zeros((n, 4*m))
    ))
    A_ineq = np.vstack((A_ineq, A))
    b_ineq = np.concatenate((b_ineq, np.ones(n)))
    # A_ineq = np.hstack((A_ineq, np.zeros((A_ineq.shape[0], 2*m))))
    #b_ineq = np.concatenate((b_ineq, b_1))
    return A_ineq, b_ineq

def parse_solution(
        supply_volumes: np.ndarray,
        supply_prices: np.ndarray,
        x_sol: np.ndarray,
        gen_names: np.ndarray
) -> pd.DataFrame:

    n, k = supply_prices.shape[0], x_sol.shape[0]
    m = k // (2*n)
    W, D = np.reshape(x_sol[:n*m], (m, n)), np.reshape(x_sol[n*m:], (m, n))
    V, P = np.tile(supply_volumes, (m, 1)), np.tile(supply_prices, (m, 1))
    W_v = W * V
    D_p = D * P
    B_v, B_p = np.zeros((m, 3)), np.zeros((m, 3))
    I = np.argsort(W_v, axis=1)
    for row_ind in range(m):
        B_v[row_ind, :] = np.flip(W_v[row_ind, I[row_ind, :]])[:3]
        B_p[row_ind, :] = np.flip(D_p[row_ind, I[row_ind, :]])[:3]
    result = pd.DataFrame(data=np.hstack((B_v, B_p)))
    result.columns = ['vol_1', 'vol_2', 'vol_3', 'price_1', 'price_2', 'price_3']
    result.insert(loc=0, column='gen_name', value=gen_names)
    return result

def calculate_alpha_coeffs(
        gen_prices: np.ndarray,
        supply_prices: np.ndarray
) -> np.ndarray:
    m, n = gen_prices.shape[0], supply_prices.shape[0]
    alpha_coeffs = np.zeros((m, n))
    for gen_index in range(m):
        alpha_coeffs[gen_index, supply_prices >= gen_prices[gen_index]] = 1
    return alpha_coeffs

In [6]:
def decompose_supply_curve(
        supply_volumes: np.ndarray,
        supply_prices: np.ndarray,
        gen_df: pd.DataFrame
) -> pd.DataFrame:
    gen_matrix = gen_df.to_numpy()[:, 1:]
    n, m = supply_volumes.shape[0], gen_matrix.shape[0]
    k = 2*n*m + 4*m
    int_indices = [(index, ) for index in range(n*m, k)]
    x = cp.Variable((k,), integer=int_indices)
    lower_bounds = np.zeros(k)
    upper_bounds = np.concatenate((np.ones(2*m*n), 1e3*np.ones(4*m)))

    A_eq, b_eq = construct_equalities(
        supply_volumes=supply_volumes,
        supply_prices=supply_prices,
        gen_matrix=gen_matrix
    )
    A_ineq, b_ineq = construct_inequalities(
        supply_volumes=supply_volumes,
        supply_prices=supply_prices,
        gen_matrix=gen_matrix
    )

    # c = np.concatenate((
    #     np.tile(supply_volumes, m),
    #     np.zeros(n*m)
    # ))
    c = np.concatenate((np.zeros(2*n*m), np.ones(4*m)))
    problem = cp.Problem(
        cp.Minimize(c @ x),
        [
            A_eq @ x == b_eq,
            A_ineq @ x <= b_ineq,
            x >= lower_bounds,
            x <= upper_bounds
        ]
    )
    problem.solve(solver=cp.MOSEK, verbose=True)
    # res = np.array(x.value[:-4*m])
    # res = np.reshape(res, (m, 2*n))
    # gen_bids = pd.DataFrame(data=res)
    gen_bids = parse_solution(
        supply_volumes=supply_volumes,
        supply_prices=supply_prices,
        x_sol=np.array(x.value[:-4*m]),
        gen_names=gen_df.loc[:, 'gen_name'].to_numpy()
    )
    return gen_bids

In [7]:
supply_prices = np.array([10, 20, 30, 40])
supply_volumes = np.array([100, 200, 300, 400])
gen_df = pd.DataFrame({
    'gen_name': np.array(['gen_1', 'gen_2']),
    'p_ats': np.array([100, 150]),
    'p_min': np.array([50, 100]),
    'p_max': np.array([200, 250]),
    'gen_price': np.array([25, 35])
})
gen_matrix = gen_df.to_numpy()[:, 1:]
A_eq, b_eq = construct_equalities(supply_volumes=supply_volumes, supply_prices=supply_prices, gen_matrix=gen_matrix)

gen_bids = decompose_supply_curve(
    supply_volumes=supply_volumes,
    supply_prices=supply_prices,
    gen_df=gen_df
)
gen_bids

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Apr 30 09:44:28 PM: Your problem has 24 variables, 4 constraints, and 0 parameters.
(CVXPY) Apr 30 09:44:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 30 09:44:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 30 09:44:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 30 09:44:28 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Apr 30 09:44:28 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

,gen_name,vol_1,vol_2,vol_3,price_1,price_2,price_3
0,gen_1,100.0,50.0,-0.0,40.0,20.0,0.0
1,gen_2,100.0,50.0,-0.0,40.0,30.0,0.0


In [8]:
import mosek